In [1]:
import numpy as np
import pythonToC
import pythonTopython2
from parsing_yacc import main as yacc_main
from parsing_lex import MyLexer

# Define test input
input_code = """
import numpy as np

a = np.array([1, 2, 3])
b = np.array([4, 5, 6])

c = a + b
d = a - b
e = np.sum(a)
f = np.dot(a, b)
"""

# Create an instance of MyLexer
lexer = MyLexer()

# Build the lexer
lexer.build()

# Test the lexer on the input code
lexer.test(input_code)

# Generate C code from Python input using yacc
print("Generating C code...")
c_output = yacc_main(input_code, 'C')
print("C code generation complete.")

print("c_output:", c_output)

if c_output:
    # Set up files for conversion
    pythonToC.set_up("test")
    pythonTopython2.setup("test")

    # Execute the input code to define variables
    exec(input_code)

    # Generate C code from Python input using pythontoC functions
    for line in input_code.split('\n'):
        if '=' in line:
            variable, value = map(str.strip, line.split("="))
            if '[' in value:  # Check if value is an array
                array_length = len(eval(value))  # Evaluate the array and get its length
                pythonToC.set_arr(variable, value, array_length, "test")
            else:  # Single value
                pythonToC.set_arr(variable, value, 1, "test")

    # Generate Python code from Python input using pythontopython functions
    for line in input_code.split('\n'):
        if '=' in line:
            exec("pythonTopython2." + line.strip())

    # Generate Python code from C input using pythontopython functions
    for line in c_output.split('\n'):
        if line.strip():
            exec("pythonTopython2." + line.strip())

    print("Conversion complete.")
else:
    print("No C output generated.")


LexToken(VARIABLE,'import',1,1)
LexToken(VARIABLE,'numpy',1,8)
LexToken(VARIABLE,'as',1,14)
LexToken(VARIABLE,'np',1,17)
LexToken(VARIABLE,'a',1,21)
LexToken(EQUALS,'=',1,23)
LexToken(ARRAY,'np.array',1,25)
LexToken(LPAREN,'(',1,33)
LexToken(LIST,'[1, 2, 3]',1,34)
LexToken(RPAREN,')',1,43)
LexToken(VARIABLE,'b',1,45)
LexToken(EQUALS,'=',1,47)
LexToken(ARRAY,'np.array',1,49)
LexToken(LPAREN,'(',1,57)
LexToken(LIST,'[4, 5, 6]',1,58)
LexToken(RPAREN,')',1,67)
LexToken(VARIABLE,'c',1,70)
LexToken(EQUALS,'=',1,72)
LexToken(VARIABLE,'a',1,74)
LexToken(PLUS,'+',1,76)
LexToken(VARIABLE,'b',1,78)
LexToken(VARIABLE,'d',1,80)
LexToken(EQUALS,'=',1,82)
LexToken(VARIABLE,'a',1,84)
LexToken(MINUS,'-',1,86)
LexToken(VARIABLE,'b',1,88)
LexToken(VARIABLE,'e',1,90)
LexToken(EQUALS,'=',1,92)
LexToken(NP_SUM,'np.sum',1,94)
LexToken(LPAREN,'(',1,100)
LexToken(VARIABLE,'a',1,101)
LexToken(RPAREN,')',1,102)
LexToken(VARIABLE,'f',1,104)
LexToken(EQUALS,'=',1,106)
LexToken(NP_DOT_PRODUCT,'np.dot',1,108)
LexTok